The objective of this notebook is to showcase the uncertainty aspect of the ClimateLearn package.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from climate_learn.utils.datetime import Year, Days, Hours
from climate_learn.data import DataModule

data_module = DataModule(
    dataset = "ERA5",
    task = "forecasting",
    root_dir = "/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg",
    in_vars = ["2m_temperature"],
    out_vars = ["2m_temperature"],
    train_start_year = Year(1979),
    val_start_year = Year(1980),
    test_start_year = Year(1981),
    end_year = Year(1982),
    history = 3,
    window = 6,
    pred_range = Days(3),
    subsample = Hours(1),
    batch_size = 128,
    num_workers = 2
)

Creating train dataset
/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg


  0%|          | 0/1 [00:00<?, ?it/s]

Creating val dataset
/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg


  0%|          | 0/1 [00:00<?, ?it/s]

Creating test dataset
/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from climate_learn.models import load_model

model_kwargs = {
    "in_channels": len(data_module.hparams.in_vars),
    "out_channels": len(data_module.hparams.out_vars),
    "n_blocks": 4,
    "history": 3,
    "dropout": 0.1,
    "prob_type": "parametric"
}

optim_kwargs = {
    "optimizer": "adam",
    "lr": 5e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": 5,
    "max_epochs": 30,
    "warmup_start_lr": 1e-8,
    "eta_min": 1e-8
}

model_module = load_model(
    name = "resnet",
    task = "forecasting",
    model_kwargs = model_kwargs,
    optim_kwargs = optim_kwargs
)

In [4]:
from climate_learn.models import set_climatology
set_climatology(model_module, data_module)

In [5]:
from climate_learn.training import Trainer

trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 1
)

In [6]:
trainer.fit(model_module, data_module)

/home/jason.jewik/miniconda3/envs/climate/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /home/jason.jewik/climate_learn/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                       ┃ Type              ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ net                        │ ResNet            │  1.2 M │
│ 1  │ net.activation             │ LeakyReLU         │      0 │
│ 2  │ net.image_proj             │ PeriodicConv2D    │ 18.9 K │
│ 3  │ net.image_proj.padding     │ PeriodicPadding2D │      0 │
│ 4  │ net.image_proj.conv        │ Conv2d            │ 18.9 K │
│ 5  │ net.blocks                 │ ModuleList        │  1.2 M │
│ 6  │ net.blocks.0               │ ResidualBlock     │  295 K │
│ 7  │ net.blocks.0.activation    │ LeakyReLU         │      0 │
│ 8  │ net.blocks.0.conv1         │ PeriodicConv2D    │  147 K │
│ 9  │ net.blocks.0.conv1.padding │ PeriodicPadding2D │      0 │
│ 10 │ net.blocks.0.conv1.conv    │ Conv2d            │  147 K │
│ 11 │ net.blocks.0.conv2         │ PeriodicConv2D    │  147 K │
│ 12 │ net.blocks.0.conv2.padding │ PeriodicPadding2D │      0 │
│ 13 │ net.blocks.0.conv2.conv    │ Conv2d            │  147 K │
│ 14 │ net.blocks.0.shortcut      │ Identity          │      0 │
│ 15 │ net.blocks.0.norm1         │ BatchNorm2d       │    256 │
│ 16 │ net.blocks.0.norm2         │ BatchNorm2d       │    256 │
│ 17 │ net.blocks.0.drop          │ Dropout           │      0 │
│ 18 │ net.blocks.1               │ ResidualBlock     │  295 K │
│ 19 │ net.blocks.1.activation    │ LeakyReLU         │      0 │
│ 20 │ net.blocks.1.conv1         │ PeriodicConv2D    │  147 K │
│ 21 │ net.blocks.1.conv1.padding │ PeriodicPadding2D │      0 │
│ 22 │ net.blocks.1.conv1.conv    │ Conv2d            │  147 K │
│ 23 │ net.blocks.1.conv2         │ PeriodicConv2D    │  147 K │
│ 24 │ net.blocks.1.conv2.padding │ PeriodicPadding2D │      0 │
│ 25 │ net.blocks.1.conv2.conv    │ Conv2d            │  147 K │
│ 26 │ net.blocks.1.shortcut      │ Identity          │      0 │
│ 27 │ net.blocks.1.norm1         │ BatchNorm2d       │    256 │
│ 28 │ net.blocks.1.norm2         │ BatchNorm2d       │    256 │
│ 29 │ net.blocks.1.drop          │ Dropout           │      0 │
│ 30 │ net.blocks.2               │ ResidualBlock     │  295 K │
│ 31 │ net.blocks.2.activation    │ LeakyReLU         │      0 │
│ 32 │ net.blocks.2.conv1         │ PeriodicConv2D    │  147 K │
│ 33 │ net.blocks.2.conv1.padding │ PeriodicPadding2D │      0 │
│ 34 │ net.blocks.2.conv1.conv    │ Conv2d            │  147 K │
│ 35 │ net.blocks.2.conv2         │ PeriodicConv2D    │  147 K │
│ 36 │ net.blocks.2.conv2.padding │ PeriodicPadding2D │      0 │
│ 37 │ net.blocks.2.conv2.conv    │ Conv2d            │  147 K │
│ 38 │ net.blocks.2.shortcut      │ Identity          │      0 │
│ 39 │ net.blocks.2.norm1         │ BatchNorm2d       │    256 │
│ 40 │ net.blocks.2.norm2         │ BatchNorm2d       │    256 │
│ 41 │ net.blocks.2.drop          │ Dropout           │      0 │
│ 42 │ net.blocks.3               │ ResidualBlock     │  295 K │
│ 43 │ net.blocks.3.activation    │ LeakyReLU         │      0 │
│ 44 │ net.blocks.3.conv1         │ PeriodicConv2D    │  147 K │
│ 45 │ net.blocks.3.conv1.padding │ PeriodicPadding2D │      0 │
│ 46 │ net.blocks.3.conv1.conv    │ Conv2d            │  147 K │
│ 47 │ net.blocks.3.conv2         │ PeriodicConv2D    │  147 K │
│ 48 │ net.blocks.3.conv2.padding │ PeriodicPadding2D │      0 │
│ 49 │ net.blocks.3.conv2.conv    │ Conv2d            │  147 K │
│ 50 │ net.blocks.3.shortcut      │ Identity          │      0 │
│ 51 │ net.blocks.3.norm1         │ BatchNorm2d       │    256 │
│ 52 │ net.blocks.3.norm2         │ BatchNorm2d       │    256 │
│ 53 │ net.blocks.3.drop          │ Dropout           │      0 │
│ 54 │ net.norm                   │ BatchNorm2d       │    256 │
│ 55 │ net.final                  │ PeriodicConv2D    │  6.3 K │
│ 56 │ net.final.padding          │ PeriodicPadding2D │      0 │
│ 57 │ net.final.conv             │ Conv2d            │  6.3 K │
│ 58 │ net.final_std              │

Trainable params: 1.2 M                                                                      
Non-trainable params: 0                                                                      
Total params: 1.2 M                                                                          
Total estimated model params size (MB): 2

Output()

/home/jason.jewik/miniconda3/envs/climate/lib/python3.8/site-packages/pytorch_lightning/train
er/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, 
does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` (try 64 which is the number of cpus on this machine) in the 
`DataLoader` init to improve performance.
  rank_zero_warn(

/home/jason.jewik/miniconda3/envs/climate/lib/python3.8/site-packages/pytorch_lightning/train
er/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, 
does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` (try 64 which is the number of cpus on this machine) in the 
`DataLoader` init to improve performance.
  rank_zero_warn(

In [7]:
trainer.test(model_module, data_module)

Output()

/home/jason.jewik/miniconda3/envs/climate/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test/acc_2m_temperature_day_3        │            -0.025567482669740012            │
│      test/w_rmse_2m_temperature_day_3       │              287.451417920544               │
│ test_climatology_baseline/w_rmse_2m_temper… │              5.891154447057514              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘